Lab 3: Language Modeling
=============
In this problem set, you will train a language model. You will:

- Train an n-gram language model.
- Use that language model to generate representative sentences.
- Study the effect of training data size, and language model complexity (n-gram size), on the modeling capacity of a language model.

- **To submit this assignment, compress the whole directory using tar, and submit the tarball ```lab2-submission.tgz``` on Compass.**

Total points: 100 points

# 0. Setup

In order to develop this assignment, you will need [python 3.6](https://www.python.org/downloads/) and the following libraries. Most if not all of these are part of [anaconda](https://www.continuum.io/downloads), so a good starting point would be to install that.

- [jupyter](http://jupyter.readthedocs.org/en/latest/install.html)
- [nosetests](https://nose.readthedocs.org/en/latest/)
- [nltk](https://www.nltk.org)

Here is some help on installing packages in python: https://packaging.python.org/installing/. You can use ```pip --user``` to install locally without sudo.

In [581]:
import sys
from importlib import reload
from collections import defaultdict
import numpy as np

In [582]:
print('My Python version')

print('python: {}'.format(sys.version))

My Python version
python: 3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [583]:
import nose
import nltk

In [584]:
print('My library versions')

print('nose: {}'.format(nose.__version__))
print('nltk: {}'.format(nltk.__version__))

My library versions
nose: 1.3.7
nltk: 3.4.5


To test whether your libraries are the right version, run:

`nosetests tests/test_environment.py`

In [585]:
! nosetests tests/test_environment.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


# 1. Training a language model

Let us first train a 3-gram language model using the language modeling toolkit: kenlm. To train a language model, all we need is a monolingual corpus, which we will get using nltk.

Total: 40 points

Let us first extract from nltk's reuters corpus, 2 corpora of 2 different domains, the food industry and the natural resources industry.

In [586]:
import nltk
nltk.download('punkt')

food = ['barley', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copra-cake''grain', 'groundnut', 'groundnut-oil', 'potato''soy-meal', 'soy-oil', 'soybean', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'veg-oil', 'wheat']
natural_resources = ['alum', 'fuel', 'gas', 'gold', 'iron-steel', 'lead', 'nat-gas', 'palladium', 'propane', 'tin', 'zinc']
corpus = nltk.corpus.reuters
food_corpus = corpus.raw(categories=food)
natr_corpus = corpus.raw(categories=natural_resources)

[nltk_data] Downloading package punkt to /Users/ab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Tokenization

Your first task is to tokenize the raw text into a list of sentences, which are in turn a list of words. No need for any other kind of preprocessing such as lowercasing.

- **Deliverable 1.1**: Complete the function `ece365lib.train.tokenize`. (5 points)
- **Test**: `nose tests/test_train.py:test_d1_1_tk`

In [587]:
from ece365lib import train
reload(train);

In [588]:
food_corpus_tk = train.tokenize_corpus(food_corpus)
natr_corpus_tk = train.tokenize_corpus(natr_corpus)

In [589]:
! nosetests tests/test_train.py:test_d1_1_tk

.
----------------------------------------------------------------------
Ran 1 test in 2.878s

OK


## Padding

Your second task is to pad your sentences with the start-of-sentence symbol and end-of-sentence symbol. These symbols are necessary to model the probability of words that usually start a sentence and those that usually end a sentence.

- **Deliverable 1.2**: Complete the function `ece365lib.train.pad`. (5 points)
- **Test**: `nosetests tests/test_train.py:test_d1_2_pad`

In [590]:
reload(train);

In [591]:
food_corpus_tk_pd = train.pad_corpus(food_corpus_tk)
natr_corpus_tk_pd = train.pad_corpus(natr_corpus_tk)

In [592]:
! nosetests tests/test_train.py:test_d1_2_pad

.
----------------------------------------------------------------------
Ran 1 test in 4.632s

OK


## Train-Test Split

Your third task is to split the corpora into train, for training the language model, and test, for testing the language model. We will go with the traditional 80% (train), 20% (test) split. The first floor(0.8*nsents) should constitute the training corpus, and the rest should constitute the test corpus.

- **Deliverable 1.3**: Complete the function `ece365lib.train.split_corpus`. (10 points)
- **Test**: `nosetests tests/test_train.py:test_d1_3_spc`

In [593]:
reload(train);

In [594]:
food_corpus_tr, food_corpus_te = train.split_corpus(food_corpus_tk_pd)
natr_corpus_tr, natr_corpus_te = train.split_corpus(natr_corpus_tk_pd)

In [595]:
! nosetests tests/test_train.py:test_d1_3_spc

.
----------------------------------------------------------------------
Ran 1 test in 3.134s

OK


## Splitting into n-grams

Your fourth task is to count n-grams in the text up to a specific order.

- **Deliverable 1.4**: Complete the function `ece365lib.train.count_ngrams`. (10 points)
- **Test**: `nosetests tests/test_train.py:test_d1_4_cn`

In [599]:
reload(train);

In [600]:
food_ngrams, food_vocab= train.count_ngrams(food_corpus_tr, 3)
 
natr_ngrams, natr_vocab = train.count_ngrams(natr_corpus_tr, 3)


In [601]:
! nosetests tests/test_train.py:test_d1_4_cn

.
----------------------------------------------------------------------
Ran 1 test in 4.293s

OK


## Estimating n-gram probability

Your last task in this part of the problem set is to estimate the n-gram probabilities p(w_i|w_{i-n+1}, w_{i-n+2}, .., w_{i-1}). For the purposes of this exercise we use maximum likelihood estimate and perform no smoothing. 

- **Deliverable 1.5**: Complete the function `ece365lib.train.estimate`. (10 points)
- **Test**: `nosetests tests/test_train.py:test_d1_5_es`

In [602]:
reload(train);

In [603]:
print(train.estimate(food_ngrams, ['palm'], ['producer', 'of']))
print(train.estimate(natr_ngrams, ['basis'], ['tested', 'the']))

0.25
0.5


In [604]:
!nosetests tests/test_train.py:test_d1_5_es

.
----------------------------------------------------------------------
Ran 1 test in 4.464s

OK


## Training a language model

For the sake of simplicity, and for the purposes of later parts in this problem set, we use nltk's lm module to train a language model.

In [606]:
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline

food_train, food_vocab = padded_everygram_pipeline(3, food_corpus_tk[:int(0.8*len(food_corpus_tk))])
natr_train, natr_vocab = padded_everygram_pipeline(3, natr_corpus_tk[:int(0.8*len(natr_corpus_tk))])

food_test = sum([['<s>'] + x + ['</s>'] for x in food_corpus_tk[int(0.8*len(food_corpus_tk)):]],[])
natr_test = sum([['<s>'] + x + ['</s>'] for x in natr_corpus_tk[int(0.8*len(natr_corpus_tk)):]],[])

food_lm = Laplace(3)
natr_lm = Laplace(3)

food_lm.fit(food_train, food_vocab)
natr_lm.fit(natr_train, natr_vocab)

One way to see what the language model learned is to see the sentences it can generate.

In [607]:
n_words = 10
print(food_lm.generate(n_words, random_seed=3))
print(natr_lm.generate(n_words, random_seed=3))

['<s>', '<s>', '<s>', 'GRAIN', 'REPORTS', 'IMPROVE', 'U.S.', 'FARM', 'OUTLOOK-ANALYSTS', 'The']
['<s>', '<s>', '<s>', 'HAITI', 'ANNOUNCES', 'FIND', 'OF', 'ORE-RICH', 'GOLD', 'FIELD']


# 2. Evaluating a language model

Next, we evaluate our language models using the perplexity measure, and draw conclusions on how switch of domains can affect the performance of a language model. Perplexity measures the language model capacity at predicting sentences in a test corpus.

Total: 20 points

- **Deliverable 2.1**: Complete the function `ece365lib.evaluate.get_perplexity`. (10 points)
- **Test**: `nosetests tests/test_train.py:test_d2_1_gp`

In [608]:
from ece365lib import evaluate
reload(evaluate);

In [609]:
# This might take some time
print(evaluate.get_perplexity(food_lm, food_test[:2500]))
print(evaluate.get_perplexity(food_lm, natr_test[:2500]))
print(evaluate.get_perplexity(natr_lm, natr_test[:2500]))
print(evaluate.get_perplexity(natr_lm, food_test[:2500]))

7585.725826961882
7582.514437208716
5399.859487187696
5536.548585338417


In [610]:
! nosetests tests/test_train.py:test_d2_1_gp

.
----------------------------------------------------------------------
Ran 1 test in 54.949s

OK


- **Deliverable 2.2**: What observations can you make on the results? Is the domain shift affecting the performance of the language model? Is that always the case? What are possible explanations? (10 points)

# 3. Data size and model complexity

Let us now see how the size of the training data size and the complexity of the model we choose affects the quality of our language model.

Total: 40 points

For this part we'd like to see the difference between a 2-gram model and a 4-gram model, when trained on 25%, 50%, 75%, and 100% of the training corpus. 

- **Deliverable 3.1**: Complete the function `ece365lib.train.vary_ngram`. (40 points)
- **Test**: `nosetests tests/test_train.py:test_d3_1_vary`

In [611]:
from ece365lib import train
reload(train);

In [612]:
n_gram_orders = [2, 3]

train_corpus = food_corpus_tk[:int(0.8*len(food_corpus_tk))]
test_corpus = food_corpus_tk[int(0.8*len(food_corpus_tk)): int(0.85*len(food_corpus_tk))]

results = train.vary_ngram(train_corpus, test_corpus, n_gram_orders)

print(results)

{2: 7688.982556947857, 3: 7726.992735978539}


In [613]:
! nosetests tests/test_train.py:test_d3_1_vary

.
----------------------------------------------------------------------
Ran 1 test in 108.639s

OK


We notice that the 3-gram language model actually performs worse than the 2-gram language model. This is due to the smal size of the training corpus. A 3-gram language model is actually too complex of a model for a small training size. If our training data was larger, we would be seeing the opposite.